# 01. 競馬データ収集

In [181]:
import re
import time
import datetime

import requests
import pandas as pd
from bs4 import BeautifulSoup

## カレンダー情報のHTMLソースを取得する

In [182]:
YEAR = 2025
MONTH = 5
CALENDAR_URL = f"https://race.netkeiba.com/top/calendar.html?year={YEAR}&month={MONTH}"

# netkeibaはBot対策をしており、User-Agentなどのヘッダーがないとアクセスを拒否されることがある
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(CALENDAR_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

calendar_html = r.text
# print(calendar_html)


## HTMLソースを解析して開催日を取得する

In [183]:

racedate_list = []

soup = BeautifulSoup(calendar_html, "html.parser") # BeautifulSoupでHTML解析
tabale_data = soup.find("table", class_="Calendar_Table") # 開催日が含まれるテーブルを取得
link_set = tabale_data.find_all("a") # テーブル内の全てのリンクを取得
racedate_list += [re.search(r"\d+$", link.get("href")).group() for link in link_set] # 各リンクのhref属性から、末尾の数字（開催日ID）を正規表現で抽出し、リストに追加

# 取得した開催日を表示
print("取得した開催日:")
for racedate in racedate_list:
    print(racedate)

取得した開催日:
20250503
20250504
20250510
20250511
20250517
20250518
20250524
20250525
20250531


## 開催レース一覧のHTMLソースを取得する

In [184]:
RACE_DATE = "20250503"
RACELIST_URL = f"https://db.netkeiba.com/race/list/{RACE_DATE}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(RACELIST_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

racelist_html = r.text
# print(racelist_html)


## HTMLソースを解析して、レースIDを取得する

In [185]:
raceId_list = []

soup = BeautifulSoup(racelist_html, "html.parser")
table_data = soup.body.find_all("dl", class_="race_top_data_info fc")

raceId_list += [
    re.search(r'^/race/\d+', link.find('a').get('href')).group().split('/')[-1]
    for link in table_data
    if link.find('a').get('title')
]

# 取得したレースIDを表示
print(f"取得したレース数:{len(raceId_list)}")
print("取得したレースID:")
for raceId in raceId_list:
    print(raceId)


取得したレース数:36
取得したレースID:
202504010101
202504010102
202504010103
202504010104
202504010105
202504010106
202504010107
202504010108
202504010109
202504010110
202504010111
202504010112
202505020301
202505020302
202505020303
202505020304
202505020305
202505020306
202505020307
202505020308
202505020309
202505020310
202505020311
202505020312
202508020301
202508020302
202508020303
202508020304
202508020305
202508020306
202508020307
202508020308
202508020309
202508020310
202508020311
202508020312


## レース情報のHTMLソースを取得する

In [186]:
RACE_ID = "202505021211"
RACE_URL = f"https://db.netkeiba.com/race/{RACE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(RACE_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

raceinfo_html = r.text
# print(raceinfo_html)


| 日本語        | カラム名（snake_case）     | 説明                                         |
| ------------- | ------------------------- | -------------------------------------------- |
| レースID      | `race_id`                 | 一意のレース識別子                           |
| 開催日        | `race_date`               | レースの開催日（例：2024-10-05）             |
| 開催回次      | `meeting_round`           | 開催の回数・日次（例：2回東京12日目）        |
| レースタイプ  | `race_type`               | レースのカテゴリ（例：3歳オープン）          |
| ラウンド      | `race_number`             | 当日レース番号（第何レースか）               |
| レース名      | `race_name`               | レースの名称（例：日本ダービー）             |
| レースクラス  | `race_class`              | 格付け（例：G1、G2、3勝クラス、未勝利など）  |
| レース場      | `racecourse_location`     | 開催競馬場（例：東京、中山、京都）           |
| コース種別    | `course_surface`          | コースの種類（芝・ダートなど）               |
| コース向き    | `course_direction`        | コースの向き（左・右・直線など）             |
| 距離          | `race_distance`           | レース距離（m）                              |
| 天候          | `weather`                 | 天候（例：晴、曇、雨）                       |
| 芝状態        | `track_condition`         | 馬場状態（例：良、稍重、重、不良）           |
| 頭数          | `entry_count`             | 出走馬数（レースの頭数）                     |

In [187]:
# --- 共通：HTML をパース ---
soup = BeautifulSoup(raceinfo_html, "html.parser")

# --- レース情報 DataFrame を作成 ---
# race_id
m = re.search(r"/race/(\d{12})/|get_race_result_horse_laptime\('(\d{12})'", raceinfo_html)
race_id = next(g for g in m.groups() if g)

# 開催日・開催回次・レースタイプ
smalltxt = soup.select_one("p.smalltxt").get_text(" ", strip=True)
# 例: '2025年6月1日 2回東京12日目 3歳オープン'
date_ja, meeting_round, race_type, *_ = smalltxt.split()
y, mth, d = map(int, re.findall(r"\d+", date_ja))
race_date = datetime.date(y, mth, d).isoformat()

# レース番号
race_number = soup.select_one("dl.racedata dt").get_text(strip=True).replace("R", "").strip()

# レース名・クラス
title_full = soup.select_one("dl.racedata h1").get_text(strip=True)
race_class = re.search(r"\(([^)]+)\)", title_full).group(1)
race_name  = re.sub(r"\([^)]*\)", "", title_full).strip()

# 開催競馬場の場所（例：東京）
racecourse_location = re.search(r"\d+回([^0-9]+?)\d+日", meeting_round).group(1)

# コース情報（例："芝左2400m" → surface="芝", direction="左", distance=2400）
# dl.racedata 内の <span> テキストから最初のスラッシュ前を使う
span_txt = soup.select_one("dl.racedata span").get_text(" ", strip=True)
course_segment = span_txt.split("/")[0].strip()  # "芝左2400m"
cm = re.search(r"(芝|ダート)(左|右|直線)?(\d+)m", course_segment)
course_surface   = cm.group(1)                        # "芝" または "ダート"
course_direction = cm.group(2) or ""                   # "左" "右" "直線" のいずれか
race_distance    = int(cm.group(3))                    # 数値部分（m）

# 天候・芝状態
weather         = re.search(r"天候\s*:\s*([^\s/]+)", span_txt).group(1)
track_condition = re.search(r"芝\s*:\s*([^\s/]+)", span_txt).group(1)

# 頭数（出走馬数）を取得
race_table = soup.find("table", class_="race_table_01")
entry_count = len(race_table.find_all("tr")) - 1 if race_table else 0  # ヘッダー行を除く


# １行分の辞書を作って DataFrame に
race_info = {
    "race_id"               : race_id,
    "race_date"             : race_date,
    "meeting_round"         : meeting_round,
    "race_type"             : race_type,
    "race_number"           : race_number,
    "race_name"             : race_name,
    "race_class"            : race_class,
    "racecourse_location"   : racecourse_location,
    "course_surface"        : course_surface,
    "course_direction"      : course_direction,
    "race_distance"         : race_distance,
    "weather"               : weather,
    "track_condition"       : track_condition,
    "entry_count"           : entry_count, 
}

race_df = pd.DataFrame([race_info])
pd.set_option('display.max_columns', None)

race_df


,race_id,race_date,meeting_round,race_type,race_number,race_name,race_class,racecourse_location,course_surface,course_direction,race_distance,weather,track_condition,entry_count
0,202505021211,2025-06-01,2回東京12日目,3歳オープン,11,第92回東京優駿,GI,東京,芝,左,2400,晴,良,18


| 日本語 | カラム名 | 説明                |
| --- | ------------------- | ----------------- |
| 着順  | `finish_position`   | 着順（レースでの順位）       |
| 枠番  | `frame_number`      | 枠番（ゲート位置）         |
| 馬番  | `horse_number`      | 馬番（レースでの番号）       |
| 馬名  | `horse_name`        | 馬の名前              |
| 性別  | `sex`               | 性別（牡・牝など）         |
| 年齢  | `age`               | 馬の年齢              |
| 斤量  | `weight_carried`    | 騎手が乗る際の斤量（kg）     |
| 騎手  | `jockey`            | 騎手の名前             |
| タイム | `race_time`         | レースタイム（例: 1:34.5） |
| 着差  | `margin`            | 着差（勝ち馬との差）        |
| 上り  | `last_3f`           | 最後の3ハロンのタイム（上り）   |
| 単勝  | `odds`              | 単勝オッズ             |
| 人気  | `popularity`        | 人気順位              |
| 馬体重 | `body_weight`       | 馬体重（+-含む場合も）      |
| 調教師 | `trainer`           | 調教師の名前            |
| 馬主  | `owner`             | 馬主の名前             |
| 賞金  | `prize_money`       | 獲得賞金（単位: 円）       |

In [188]:
# --- 共通：HTML をパース ---
soup = BeautifulSoup(raceinfo_html, "html.parser")

# ---  出走馬情報 DataFrame を作成 --- 
race_table = soup.find("table", class_="race_table_01")
body_rows = race_table.find_all("tr")[1:] if race_table else []

horse_records: list[dict] = []
for tr in body_rows:
    cells = [td.get_text(strip=True) for td in tr.find_all("td")]
    # 列数不足分を空文字で埋める（最大21列想定）
    cells.extend([""] * (21 - len(cells)))

    # 性齢を分割
    age_match = re.search(r"\d+", cells[4])
    age = age_match.group() if age_match else ""
    sex = cells[4].replace(age, "")

    # horse_id を馬名リンクから取得
    link = tr.find("a", href=re.compile(r"^/horse/\d+/"))
    horse_id = ""
    if link:
        m = re.search(r"/horse/(\d+)/", link["href"])
        if m:
            horse_id = m.group(1)

    rec = {
        "race_id"        : race_id,       # レース情報との紐付け用
        "finish_position": cells[0],
        "frame_number"   : cells[1],
        "horse_number"   : cells[2],
        "horse_name"     : cells[3],
        "horse_id"       : horse_id,
        "sex"            : sex,
        "age"            : age,
        "weight_carried" : cells[5],
        "jockey"         : cells[6],
        "race_time"      : cells[7],
        "margin"         : cells[8],
        "last_3f"        : cells[11],
        "odds"           : cells[12],
        "popularity"     : cells[13],
        "body_weight"    : cells[14],
        "trainer"        : cells[18],
        "owner"          : cells[19],
        "prize_money"    : cells[20],
    }
    horse_records.append(rec)

horse_df = pd.DataFrame(horse_records, columns=[
    "race_id", "finish_position", "frame_number", "horse_number", "horse_name", "horse_id", 
    "sex", "age", "weight_carried", "jockey", "race_time", "margin",
    "last_3f", "odds", "popularity", "body_weight", "trainer", "owner", "prize_money",
])

pd.set_option('display.max_columns', None)
horse_df

,race_id,finish_position,frame_number,horse_number,horse_name,horse_id,sex,age,weight_carried,jockey,race_time,margin,last_3f,odds,popularity,body_weight,trainer,owner,prize_money
0,202505021211,1,7,13,クロワデュノール,2022105102,牡,3,57,北村友一,2:23.7,,34.2,2.1,1,504(+4),[西]斉藤崇史,サンデーレーシング,"32,771.3"
1,202505021211,2,8,17,マスカレードボール,2022105519,牡,3,57,坂井瑠星,2:23.8,3/4,33.7,6.8,3,466(+6),[東]手塚貴久,社台レースホース,"12,791.8"
2,202505021211,3,1,2,ショウヘイ,2022104714,牡,3,57,ルメール,2:24.0,1.1/2,34.3,14.4,6,460(+4),[西]友道康夫,石川達絵,"7,895.9"
3,202505021211,4,8,18,サトノシャイニング,2022101420,牡,3,57,武豊,2:24.1,クビ,34.7,12.3,5,496(0),[西]杉山晴紀,里見治,"4,500.0"
4,202505021211,5,2,3,エリキング,2022105099,牡,3,57,川田将雅,2:24.3,1.1/4,33.4,17.0,8,500(0),[西]中内田充,藤田晋,"3,000.0"
5,202505021211,6,4,7,ミュージアムマイル,2022105081,牡,3,57,レーン,2:24.4,クビ,34.1,5.7,2,496(-4),[西]高柳大輔,サンデーレーシング,
6,202505021211,7,4,8,エムズ,2022105105,牡,3,57,戸崎圭太,2:24.4,クビ,34.5,77.5,11,448(+6),[西]池江泰寿,エムズレーシング,
7,202505021211,8,5,9,ジョバンニ,2022103995,牡,3,57,松山弘平,2:24.5,クビ,34.3,15.9,7,480(-2),[西]杉山晴紀,ＫＲジャパン,
8,202505021211,9,8,16,ファイアンクランツ,2022104720,牡,3,57,佐々木大,2:24.6,3/4,34.9,114.7,14,456(+2),[東]堀宣行,サンデーレーシング,
9,202505021211,10,1,1,リラエンブレム,2022104922,牡,3,57,浜中俊,2:24.7,1/2,34.4,76.9,10,484(+4),[西]武幸四郎,Ｇリビエール・レーシング,


## HTMLソースを解析して、馬IDを取得する

In [189]:
horseId_list = []

soup = BeautifulSoup(raceinfo_html, "html.parser")

# 出馬表のテーブル内だけから馬リンクを探す
horse_links = soup.select("table.race_table_01 a[href^='/horse/']")

# 正規表現で ID 部分だけ取り出す
horseId_list = [
    re.search(r"/horse/(\d+)/", a["href"]).group(1)
    for a in horse_links
]

# 重複を除去（同じ馬が複数回リンクされているケースに備えて）
horseId_list = list(dict.fromkeys(horseId_list))

# 取得した馬IDを表示
print(f"取得した馬数:{len(horseId_list)}")
print("取得した馬ID:")
for horseId in horseId_list:
    print(horseId)


取得した馬数:18
取得した馬ID:
2022105102
2022105519
2022104714
2022101420
2022105099
2022105081
2022105105
2022103995
2022104720
2022104922
2022104845
2022104896
2022104397
2022105836
2022105891
2022106155
2022104416
2022104218


## 馬情報のHTMLソースを取得する

In [190]:
HORSE_ID = "2022105102"
HORSE_URL = f"https://db.netkeiba.com/horse/{RACE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(HORSE_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

horseinfo_html = r.text
# print(horseinfo_html)


## 馬戦績のHTMLソースを取得する

In [191]:
HORSE_ID = "2022105102"
HORSE_RESULT_URL = f"https://db.netkeiba.com/horse/result/{RACE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(HORSE_RESULT_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

horseresult_html = r.text
# print(horseresult_html)

## 馬血統のHTMLソースを取得する

In [192]:
HORSE_ID = "2022105102"
HORSE_PED_URL = f"https://db.netkeiba.com/horse/ped/{RACE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(HORSE_PED_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

horseped_html = r.text
# print(horseped_html)